In [26]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pickle

In [28]:
from pathlib import Path

file_path = Path("/Users/vivek/anaconda_projects/Innomatics/data science/NLP/Next word prediction/en_US.twitter.txt")

MAX_LINES = 3_000
with file_path.open("r", encoding="utf-8") as f:
    dataset = []
    for line in f:
        clean = line.encode("ascii", "ignore").decode("ascii").strip()
        if clean:
            dataset.append(clean)
        if len(dataset) >= MAX_LINES:
            break

In [30]:
dataset

['How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.',
 "When you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.",
 "they've decided its more fun if I don't.",
 'So Tired D; Played Lazer Tag & Ran A LOT D; Ughh Going To Sleep Like In 5 Minutes ;)',
 'Words from a complete stranger! Made my birthday even better :)',
 'First Cubs game ever! Wrigley field is gorgeous. This is perfect. Go Cubs Go!',
 'i no! i get another day off from skool due to the wonderful snow (: and THIS wakes me up...damn thing',
 "I'm coo... Jus at work hella tired r u ever in cali",
 'The new sundrop commercial ...hehe love at first sight',
 'we need to reconnect THIS WEEK',
 'I always wonder how the guys on the auctions shows learned to talk so fast!? all I hear is djsosnekspqnslanskam.',
 'Dammnnnnn what a catch',
 'such a great picture! The green shirt totally brings out your eyes!',
 'Desk put tog

In [63]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=50000, lower=True)
tokenizer.fit_on_texts(dataset)

In [65]:
print(tokenizer)

In [67]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'i': 3,
 'a': 4,
 'you': 5,
 'and': 6,
 'in': 7,
 'for': 8,
 'of': 9,
 'is': 10,
 'it': 11,
 'my': 12,
 'on': 13,
 'that': 14,
 'be': 15,
 'me': 16,
 'so': 17,
 'at': 18,
 'with': 19,
 'this': 20,
 'your': 21,
 'are': 22,
 'have': 23,
 'just': 24,
 'we': 25,
 'up': 26,
 'all': 27,
 'but': 28,
 'was': 29,
 'not': 30,
 "i'm": 31,
 'like': 32,
 'what': 33,
 'love': 34,
 'out': 35,
 'do': 36,
 'get': 37,
 'good': 38,
 'day': 39,
 'one': 40,
 'if': 41,
 'about': 42,
 'know': 43,
 'will': 44,
 'thanks': 45,
 "it's": 46,
 'can': 47,
 'rt': 48,
 "don't": 49,
 'when': 50,
 'now': 51,
 'u': 52,
 'no': 53,
 'they': 54,
 'from': 55,
 'as': 56,
 'how': 57,
 'new': 58,
 'time': 59,
 'go': 60,
 'great': 61,
 'some': 62,
 'or': 63,
 'lol': 64,
 'would': 65,
 'more': 66,
 'going': 67,
 'got': 68,
 'by': 69,
 'today': 70,
 'an': 71,
 '3': 72,
 'there': 73,
 'see': 74,
 'want': 75,
 'need': 76,
 'back': 77,
 'he': 78,
 'much': 79,
 'well': 80,
 'tonight': 81,
 'really': 82,
 'who': 

In [69]:
len(tokenizer.word_index)

7384

In [71]:
print(len(tokenizer.word_index))
print(list(tokenizer.word_index.items())[:10])

7384
[('the', 1), ('to', 2), ('i', 3), ('a', 4), ('you', 5), ('and', 6), ('in', 7), ('for', 8), ('of', 9), ('is', 10)]


In [73]:
tokenized_sentences = tokenizer.texts_to_sequences(dataset)

In [75]:
print(tokenized_sentences[0])

[57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7, 1007, 624, 161, 34, 2, 74, 5, 101, 118, 118, 85, 168]


In [77]:
# for tokenized_sentence in tokenized_sentences[:5]:
#     print(tokenized_sentence)

In [79]:
input_sequences = []
for tokenized_sentence in tokenized_sentences:
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])

In [81]:
input_sequences

[[57, 22],
 [57, 22, 5],
 [57, 22, 5, 719],
 [57, 22, 5, 719, 45],
 [57, 22, 5, 719, 45, 8],
 [57, 22, 5, 719, 45, 8, 1],
 [57, 22, 5, 719, 45, 8, 1, 48],
 [57, 22, 5, 719, 45, 8, 1, 48, 5],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7, 1007],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7, 1007, 624],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7, 1007, 624, 161],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7, 1007, 624, 161, 34],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7, 1007, 624, 161, 34, 2],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7, 1007, 624, 161, 34, 2, 74],
 [57, 22, 5, 719, 45, 8, 1, 48, 5, 217, 15, 7, 1007, 624, 161, 34, 2, 74, 5],
 [57,
  22,
  5,
  719,
  45,
  8,
  1,
  48,
  5,
  217,
  15,
  7,
  1007,
  624,
  161,
  34,
  2,
  74,
  5,
  101],
 [57,
  22,
  5,
  719,
  45,
  8,
  1,
  48,
  5,
  217,
  15,
  7,
  1007,
  624,
 

In [83]:
len(input_sequences)

35246

In [85]:
max_length = max([len(x) for x in input_sequences])
max_length

34

In [87]:
padded_input_sequences = pad_sequences(input_sequences, maxlen= max_length, padding= 'pre')

In [89]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,   57,   22],
       [   0,    0,    0, ...,   57,   22,    5],
       [   0,    0,    0, ...,   22,    5,  719],
       ...,
       [   0,    0,    0, ...,  131,   25,  199],
       [   0,    0,    0, ...,   25,  199,  170],
       [   0,    0,    0, ...,  199,  170, 2632]], dtype=int32)

In [91]:
X = padded_input_sequences[:, :-1]

In [93]:
X.shape

(35246, 33)

In [95]:
Y = padded_input_sequences[:, -1]

In [97]:
Y = to_categorical(Y, num_classes=7385)

In [99]:
Y.shape

(35246, 7385)

In [101]:
model = Sequential()
model.add(Embedding(7385, 100))
model.add(LSTM(150))
model.add(Dense(7385, activation='softmax'))

In [103]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [105]:
model.build((None, 33))

In [107]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 33, 100)        │       738,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7385)           │     1,115,135 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,004,235 (7.65 MB)

 Trainable params: 2,004,235 (7.65 MB)

 Non-trainable params: 0 (0.00 B)

In [109]:
model.fit(X,Y,epochs=40)

Epoch 1/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 22ms/step - accuracy: 0.0317 - loss: 7.3553
Epoch 2/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.0446 - loss: 6.8423
Epoch 3/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 24s 22ms/step - accuracy: 0.0609 - loss: 6.5054
Epoch 4/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 24s 22ms/step - accuracy: 0.0772 - loss: 6.1449
Epoch 5/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.0930 - loss: 5.7546
Epoch 6/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.1091 - loss: 5.3500
Epoch 7/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.1362 - loss: 4.9432
Epoch 8/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.1760 - loss: 4.5459
Epoch 9/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.2298 - loss: 4.1576
Epoch 10/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.2851 - loss: 3.7840
Epoch 11/40
1102/1102 ━━━━━━━━━━━━━━━━━━━━ 25s 23ms/step - accuracy: 0.3438 - loss: 3.4330
Epoch 12

In [111]:
test_text = "i am so"

In [113]:
token_test_text = tokenizer.texts_to_sequences([test_text])[0]
token_test_text

[3, 91, 17]

In [115]:
# padding
padded_token_test_text = pad_sequences([token_test_text], maxlen=max_length, padding='pre')
print(padded_token_test_text)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  3 91 17]]


In [117]:
# predict
predict = model.predict(padded_token_test_text)
prediction = np.argmax(predict)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step


220

In [119]:
for word, index in tokenizer.word_index.items():
    if prediction == index:
        print(word)

excited


In [121]:
# Save model
model.save("next_word_model.h5")
print("Saved model to next_word_model.h5")

Saved model to next_word_model.h5


In [123]:
# Save tokenizer
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
print("Saved tokenizer to tokenizer.pkl")

Saved tokenizer to tokenizer.pkl
